<a href="https://colab.research.google.com/github/robsworld718/Housing-Sales-advanced-regression-techniques/blob/main/Advanced_Regression_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

#Loading and Cleaning

In [ ]:
sales_df = pd.read_csv('/content/kaggle_reg_train.csv', index_col='Id')
sales_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
print('Number of Duplicated Rows:', sales_df.duplicated().sum())
print('Number of NA Rows:', sales_df.isnull().sum().sum())
print('\n')
print('Data Information...')
print(sales_df.info())

Number of Duplicated Rows: 0
Number of NA Rows: 6965


Data Information...
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 n

#Feature Engineering

In [ ]:
#Combining all area columns, dropping and replacing with a "TotalArea" column
sales_df['TotalArea'] = sales_df['LotArea'] + sales_df['MasVnrArea'] + sales_df['GrLivArea']\
+ sales_df['GarageArea'] 

sales_df = sales_df.drop(['LotArea', 'MasVnrArea', 'GrLivArea', 'GarageArea'], axis=1)

In [ ]:
#Combining all area columns, dropping and replacing with a "TotalSF" column
sales_df['TotalSF'] = sales_df['TotalBsmtSF'] + sales_df['1stFlrSF'] + sales_df['2ndFlrSF']\
+ sales_df['LowQualFinSF'] + sales_df['WoodDeckSF'] + sales_df['OpenPorchSF']

sales_df = sales_df.drop(['BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
                          'WoodDeckSF', 'OpenPorchSF'], axis=1)

In [ ]:
#Concatenating 2 datetime columns into 1
sales_df['YrMoSold'] = pd.to_datetime(sales_df.YrSold.astype(str) + '/' + sales_df.MoSold.astype(str))
sales_df = sales_df.drop(['YrSold', 'MoSold'], axis=1)
sales_df.head()

,MSSubClass,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolQC,Fence,MiscFeature,MiscVal,SaleType,SaleCondition,SalePrice,TotalArea,TotalSF,YrMoSold
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,NaN,NaN,NaN,0,WD,Normal,208500,10904.0,2627,2008-02-01
2,20,RL,80.0,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,...,NaN,NaN,NaN,0,WD,Normal,181500,11322.0,2822,2007-05-01
3,60,RL,68.0,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,NaN,NaN,NaN,0,WD,Normal,223500,13806.0,2748,2008-09-01
4,70,RL,60.0,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,...,NaN,NaN,NaN,0,WD,Abnorml,140000,11909.0,2508,2006-02-01
5,60,RL,84.0,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,...,NaN,NaN,NaN,0,WD,Normal,250000,17644.0,3619,2008-12-01


In [ ]:
sales_df['YrMoSold'] = sales_df['YrMoSold'].astype('object')

In [ ]:
#Checking the data balance
sales_df["HouseStyle"].value_counts(normalize=True)

1Story    0.497260
2Story    0.304795
1.5Fin    0.105479
SLvl      0.044521
SFoyer    0.025342
1.5Unf    0.009589
2.5Unf    0.007534
2.5Fin    0.005479
Name: HouseStyle, dtype: float64

#Pre-Processing

In [ ]:
from seaborn import heatmap
from sklearn import set_config
set_config(display='diagram')
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression

In [ ]:
X = sales_df.drop(columns=['SalePrice'])
y = sales_df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

scaler = StandardScaler()
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

mean_imputer = SimpleImputer(strategy = 'mean')
freq_imputer = SimpleImputer(strategy = 'most_frequent')

num_pipe = make_pipeline(mean_imputer, scaler)
cat_pipe = make_pipeline(freq_imputer, ohe_encoder)

num_tuple = (num_pipe, num_selector)
cat_tuple = (cat_pipe, cat_selector)

In [ ]:
col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder = 'passthrough')

In [ ]:
X_train_processed = col_transformer.fit(X_train)

In [ ]:
col_transformer.named_transformers_

{'pipeline-1': Pipeline(steps=[('simpleimputer', SimpleImputer()),
                 ('standardscaler', StandardScaler())]),
 'pipeline-2': Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse=False))])}

In [ ]:
#Creating a variable for cat feature columns through its respective pipeline
cat_feature_names = col_transformer.named_transformers_['pipeline-2']\
                          .named_steps['onehotencoder']\
                          .get_feature_names_out(cat_selector(X_train))

cat_feature_names

array(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl',
       'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3',
       'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS',
       'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub',
       'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac',
       'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside',
       'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev',
       'Neighborhood_Blmngtn', 'Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NPkVill',
       'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_

In [ ]:
final_cols = num_selector(X_train) + list(cat_feature_names)
final_cols

['MSSubClass',
 'LotFrontage',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'TotalArea',
 'TotalSF',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Grvl',
 'Street_Pave',
 'Alley_Grvl',
 'Alley_Pave',
 'LotShape_IR1',
 'LotShape_IR2',
 'LotShape_IR3',
 'LotShape_Reg',
 'LandContour_Bnk',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_Lvl',
 'Utilities_AllPub',
 'Utilities_NoSeWa',
 'LotConfig_Corner',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'LandSlope_Gtl',
 'LandSlope_Mod',
 'LandSlope_Sev',
 'Neighborhood_Blmngtn',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighbor

In [ ]:
final_cols[0].split('__')[-1]

clean_names = []
for col in final_cols:
  name = col.split('__')[-1]
  clean_names.append(name)

In [ ]:
#Transforming the X_train and X_test into an array
X_train_transformed = col_transformer.transform(X_train)

X_test_transformed = col_transformer.transform(X_test)

In [ ]:
#Checking to see if all missing values were filled
print(np.isnan(X_train_transformed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_transformed).sum().sum(), 'missing values in testing data')

0 missing values in training data
0 missing values in testing data


In [ ]:
#Converted both into individual dataframes all data has been encoded and scaled to apply to our models
X_train_output = pd.DataFrame(X_train_transformed, columns = final_cols)

X_test_output = pd.DataFrame(X_test_transformed, columns = final_cols)

#Creating Models

Baseline Model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(y_true, y_pred):
  """takes two arrays, true labels and predicted labels, and prints
  MAE, MSE, RMSE, and R2 metrics"""
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  r2 = r2_score(y_true, y_pred)
  rmse = np.sqrt(mse)
  print(f'MAE {mae}\n MSE {mse}\n R^2 {r2}\n RMSE {rmse}')

In [ ]:
#Creating and evaluating a baseline
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor(strategy="mean")

dummy_pipe = make_pipeline(col_transformer, dummy)

dummy_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc749d2f050>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc749d2f090>)])),
                ('dummyregressor', DummyRegressor())])

In [ ]:
train_dummy_preds = dummy_pipe.predict(X_train)
test_dummy_preds = dummy_pipe.predict(X_test)

In [ ]:
train_score = dummy_pipe.score(X_train, y_train)
test_score = dummy_pipe.score(X_test, y_test)
print('Training Metrics\n')
evaluate_model(y_train, train_dummy_preds)
print('\n')

print('Testing Metrics\n')
evaluate_model(y_test, test_dummy_preds)

Training Metrics

MAE 56797.487783824356
 MSE 6071445130.642884
 R^2 0.0
 RMSE 77919.47850597362


Testing Metrics

MAE 60115.176811159065
 MSE 7015322248.511615
 R^2 -0.0014293725084049136
 RMSE 83757.52054897288


Randomn Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 42)

rf_pipe = make_pipeline(col_transformer, rf)

In [ ]:
rf_pipe.get_params()

{'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('pipeline-1',
                                  Pipeline(steps=[('simpleimputer',
                                                   SimpleImputer()),
                                                  ('standardscaler',
                                                   StandardScaler())]),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc749d2f050>),
                                 ('pipeline-2',
                                  Pipeline(steps=[('simpleimputer',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('onehotencoder',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse=False))]),
                                  <skle

In [ ]:
rf_param_grid = {
    'randomforestregressor__max_depth': (4, 8),
    'randomforestregressor__n_estimators': (500, 1000),
    'randomforestregressor__min_samples_leaf': [3,15],   
}

In [ ]:
rf_pipe_gs = GridSearchCV(rf_pipe, rf_param_grid)

In [ ]:
%%time
rf_pipe_gs.fit(X_train, y_train)
print('Best RF Parameters:')
print(rf_pipe_gs.best_params_)
best_pipe = rf_pipe_gs.best_estimator_
print(f'R^2 of best RF model is: {best_pipe.score(X_test, y_test)}')

Best RF Parameters:
{'randomforestregressor__max_depth': 8, 'randomforestregressor__min_samples_leaf': 3, 'randomforestregressor__n_estimators': 500}
R^2 of best RF model is: 0.8849705355060077
CPU times: user 4min 4s, sys: 774 ms, total: 4min 5s
Wall time: 4min 12s


In [ ]:
best_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc74920c950>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc74920cfd0>)])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=8, min_samples_leaf=3,
                                       n_estimators=500, random_state=42))])

In [ ]:
train_rf_preds = best_pipe.predict(X_train)
test_rf_preds = best_pipe.predict(X_test)

best_pipe.predict(X_test)

array([145181.95661117, 331280.2068657 , 120335.32953135, 153173.87931544,
       333860.88750167,  84301.6996713 , 202444.34451629, 151263.95878779,
        84238.10448086, 135827.58406746, 139552.81692233, 120014.11320589,
       108391.91837276, 209175.00925113, 180667.65254376, 124929.6777314 ,
       184633.31551457, 130025.02994625, 116268.54598246, 213955.50913116,
       170737.67856896, 198761.53868368, 177507.75200544, 120295.68320901,
       202945.01175472, 172005.97160736, 210009.64108052, 113111.42179416,
       183724.2004931 , 206331.82893996, 122403.65450663, 236462.65664893,
       217581.35267049, 115772.92421111, 261567.04726367, 156125.70477871,
       143441.8794811 , 197899.46239296, 319999.91086417, 104913.51626231,
       128699.20680973, 220495.58937334, 119452.48031639, 369274.41397427,
       130424.72004474, 161322.58061542, 115916.24492461, 123082.43038387,
       414702.97020006, 136605.25962073, 119177.28463385, 198089.75100316,
       119512.2009995 , 3

In [ ]:
train_score = best_pipe.score(X_train, y_train)
test_score = best_pipe.score(X_test, y_test)
print('Training Metrics\n')
evaluate_model(y_train, train_rf_preds)
print('\n')

print('Testing Metrics\n')
evaluate_model(y_test, test_rf_preds)

Training Metrics

MAE 10997.949105796004
 MSE 338740999.67130643
 R^2 0.9442075169284387
 RMSE 18404.917812131258


Testing Metrics

MAE 17054.261606647055
 MSE 805816948.905509
 R^2 0.8849705355060077
 RMSE 28386.915100192007


XGBoost Model

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb_pipe = make_pipeline(col_transformer, xgb)
xgb_pipe.fit(X_train, y_train)

[06:55:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc749d2f050>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc749d2f090>)])),
                ('xgbregressor', XGBRegressor())])

In [ ]:
xgb_pipe.get_params()

{'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('pipeline-1',
                                  Pipeline(steps=[('simpleimputer',
                                                   SimpleImputer()),
                                                  ('standardscaler',
                                                   StandardScaler())]),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc749d2f050>),
                                 ('pipeline-2',
                                  Pipeline(steps=[('simpleimputer',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('onehotencoder',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse=False))]),
                                  <skle

In [ ]:
xgb_param_grid = {
    'xgbregressor__max_depth': (4, 8),
    'xgbregressor__n_estimators': (500, 1000),
    'xgbregressor__min_samples_leaf': [3,15],
    'xgbregressor__booster': ('gbtree', 'gblinear')   
}

In [ ]:
xgb_pipe_gs = GridSearchCV(xgb_pipe, xgb_param_grid)

In [ ]:
%%time
xgb_pipe_gs.fit(X_train, y_train)
print('Best RF Parameters:')
print(xgb_pipe_gs.best_params_)
best_pipe2 = xgb_pipe_gs.best_estimator_
print(f'R^2 of best XGB model is: {best_pipe2.score(X_test, y_test)}')

[06:55:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:56:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
best_pipe2.fit(X_train, y_train)

[07:03:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc74929d590>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fc74929dcd0>)])),
                ('xgbregressor',
                 XGBRegressor(max_depth=4, min_samples_leaf=3,
                              n_estimators=500))])

In [ ]:
train_xgb_preds = best_pipe2.predict(X_train)
test_xgb_preds = best_pipe2.predict(X_test)

best_pipe2.predict(X_test)

array([147007.14 , 357867.2  , 108647.35 , 152000.36 , 331399.72 ,
        79455.49 , 218492.69 , 148353.36 ,  81348.41 , 143333.22 ,
       143347.4  , 120105.96 ,  98301.06 , 195808.2  , 175588.3  ,
       130544.484, 185424.6  , 125412.805, 118583.18 , 220776.2  ,
       161335.23 , 223079.83 , 168964.73 , 122347.19 , 202058.34 ,
       159961.06 , 196519.94 , 112952.84 , 177462.66 , 193105.92 ,
       119041.84 , 246378.72 , 233448.7  , 124989.195, 284106.97 ,
       153680.47 , 136788.11 , 203863.6  , 310764.28 , 111154.06 ,
       140682.17 , 231713.77 , 116239.76 , 352767.44 , 129437.46 ,
       135667.48 , 109632.15 , 121466.85 , 427204.53 , 128442.44 ,
       119667.95 , 199411.52 , 112851.984, 281044.12 , 148404.77 ,
       229074.05 , 199830.25 , 155860.7  , 142814.62 ,  93413.16 ,
        77235.23 , 160677.11 , 312089.34 , 266322.4  , 335950.47 ,
       224787.1  , 121567.58 , 323693.22 , 123487.35 , 168715.44 ,
       117545.99 , 129775.85 , 110066.62 ,  84011.87 , 503648.

XGB is the best performing model, so now we can test it on the testing data

In [ ]:
train_score = best_pipe2.score(X_train, y_train)
test_score = best_pipe2.score(X_test, y_test)
print('Training Metrics\n')
evaluate_model(y_train, train_xgb_preds)
print('\n')

print('Testing Metrics\n')
evaluate_model(y_test, test_xgb_preds)

Training Metrics

MAE 2996.7970034246578
 MSE 17014360.24811549
 R^2 0.9971976424258134
 RMSE 4124.846693892452


Testing Metrics

MAE 15746.264661815068
 MSE 584156052.3254688
 R^2 0.9166123795594145
 RMSE 24169.32047711455


#Submission

We need to apply the same cleaning and pre-processing steps to the test set

In [ ]:
sales_test_df = pd.read_csv('/content/kaggle_reg_train.csv')
sales_test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
#Combining all area columns, dropping and replacing with a "TotalArea" column
sales_test_df['TotalArea'] = sales_test_df['LotArea'] + sales_test_df['MasVnrArea'] + sales_test_df['GrLivArea']\
+ sales_test_df['GarageArea'] 

sales_test_df = sales_test_df.drop(['LotArea', 'MasVnrArea', 'GrLivArea', 'GarageArea'], axis=1)

In [ ]:
#Combining all area columns, dropping and replacing with a "TotalSF" column
sales_test_df['TotalSF'] = sales_test_df['TotalBsmtSF'] + sales_test_df['1stFlrSF'] + sales_test_df['2ndFlrSF']\
+ sales_test_df['LowQualFinSF'] + sales_test_df['WoodDeckSF'] + sales_test_df['OpenPorchSF']

sales_test_df = sales_test_df.drop(['BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
                          'WoodDeckSF', 'OpenPorchSF'], axis=1)

In [ ]:
#Concatenating 2 datetime columns into 1
sales_test_df['YrMoSold'] = pd.to_datetime(sales_test_df.YrSold.astype(str) + '/' + sales_test_df.MoSold.astype(str))
sales_test_df = sales_test_df.drop(['YrSold', 'MoSold'], axis=1)
sales_test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolQC,Fence,MiscFeature,MiscVal,SaleType,SaleCondition,SalePrice,TotalArea,TotalSF,YrMoSold
0,1,60,RL,65.0,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,NaN,0,WD,Normal,208500,10904.0,2627,2008-02-01
1,2,20,RL,80.0,Pave,NaN,Reg,Lvl,AllPub,FR2,...,NaN,NaN,NaN,0,WD,Normal,181500,11322.0,2822,2007-05-01
2,3,60,RL,68.0,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,NaN,NaN,0,WD,Normal,223500,13806.0,2748,2008-09-01
3,4,70,RL,60.0,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,NaN,NaN,0,WD,Abnorml,140000,11909.0,2508,2006-02-01
4,5,60,RL,84.0,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,NaN,NaN,0,WD,Normal,250000,17644.0,3619,2008-12-01


In [ ]:
sales_test_df['YrMoSold'] = sales_test_df['YrMoSold'].astype('object')

In [ ]:
cat_selector2 = make_column_selector(dtype_include='object')
num_selector2 = make_column_selector(dtype_include='number')

scaler2 = StandardScaler()
ohe_encoder2 = OneHotEncoder(sparse=False, handle_unknown='ignore')

mean_imputer2 = SimpleImputer(strategy = 'mean')
freq_imputer2 = SimpleImputer(strategy = 'most_frequent')

num_pipe2 = make_pipeline(mean_imputer2, scaler)
cat_pipe2 = make_pipeline(freq_imputer2, ohe_encoder)

num_tuple2 = (num_pipe, num_selector)
cat_tuple2 = (cat_pipe, cat_selector)

In [ ]:
col_transformer2 = make_column_transformer(num_tuple2, cat_tuple2, remainder = 'passthrough')

In [ ]:
sales_test_processed = col_transformer2.fit(sales_test_df)

In [ ]:
col_transformer2.named_transformers_

{'pipeline-1': Pipeline(steps=[('simpleimputer', SimpleImputer()),
                 ('standardscaler', StandardScaler())]),
 'pipeline-2': Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse=False))])}

In [ ]:
#Creating a variable for cat feature columns through its respective pipeline
cat_feature_names2 = col_transformer2.named_transformers_['pipeline-2']\
                          .named_steps['onehotencoder']\
                          .get_feature_names_out(cat_selector(X_train))

In [ ]:
final_cols2 = num_selector2(X_train) + list(cat_feature_names)

In [ ]:
final_cols2[0].split('__')[-1]

clean_names2 = []
for col in final_cols:
  name = col.split('__')[-1]
  clean_names2.append(name)

In [ ]:
#Transforming the test dataframe into an array
sales_test_df_transformed2 = col_transformer.transform(sales_test_df)

In [ ]:
#Checking to see if all missing values were filled
print(np.isnan(sales_test_df_transformed2).sum().sum(), 'missing values in testing data')

0 missing values in testing data


In [ ]:
df_test = col_transformer2.transform(sales_test_df)

In [ ]:
#New Predications
preds = best_pipe2.predict(X)

In [ ]:
#creat new DataFrame with results of the modeling
new_data = pd.DataFrame(preds)
new_data.columns=['SalePrice']
new_data.head()

,SalePrice
0,205132.109375
1,181411.875000
2,220883.703125
3,144114.812500
4,253014.031250


In [ ]:
#concat two DataFrames
submission = pd.concat((sales_test_df['Id'], new_data), axis=1)
submission.set_index('Id')
submission

,Id,SalePrice
0,1,205132.109375
1,2,181411.875000
2,3,220883.703125
3,4,144114.812500
4,5,253014.031250
...,...,...
1455,1456,176734.171875
1456,1457,209960.140625
1457,1458,265136.031250
1458,1459,148852.562500


In [ ]:
submission['Id'] = sales_test_df.index

In [ ]:
submission.isna().sum()

Id           0
SalePrice    0
dtype: int64

In [ ]:
#save to csv file
submission.to_csv('/content/drive/MyDrive/submission.csv', index = False, header=True)

In [ ]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1460 non-null   int64  
 1   SalePrice  1460 non-null   float32
dtypes: float32(1), int64(1)
memory usage: 17.2 KB


In [ ]:
#print image of submission
from IPython.display import Image
Image('/content/drive/MyDrive/Bootcamp/11 Week_ Introduction to Deep Learning/Kaggle.jpg')